The following code was tested only with Linux OS. The Oracle DB client was installed and the cx_Oracle python library was used with Python 3.5.

In [1]:
import cx_Oracle
import csv
import os

We set the language to UTF8 so we can import and read Unicode characters that exist in the dataset.

In [2]:
os.environ['NLS_LANG'] = '.AL32UTF8'
db = cx_Oracle.connect('user_1', 'babis', '52.50.54.24:1521/ORCL', encoding='utf-8')
print('Oracle DB server version %s' % db.version)

Oracle DB server version 12.1.0.2.0


Create the connection cursor we will use for all operations.

In [3]:
cursor = db.cursor()

We read the SQL script and process it command by command.

In [9]:
script_file = open('part3/Part 3.sql')
script = script_file.read()
commands = script.split(';')

for command in commands:
    if(command.isspace()==False and len(command) > 0):
        command = command.strip()
        cursor.execute(command)

Since we examine the cases with up to 3 hops (streets) the folowing query will do. It examines the available companies at each number of hops starting from AUEB.

In [10]:
part3a = ('SELECT B AS company, d AS distance '
          'FROM streets '
          'WHERE A=\'AUEB\' '
          'AND d <= 10 '
          'UNION '
          'SELECT street2.B AS company, (street1.d+street2.d) AS distance '
          'FROM streets street1 '
          'JOIN streets street2 '
          'ON street1.B=street2.A '
          'WHERE street1.A=\'AUEB\' '
          'AND street1.d+street2.d <= 10 '
          'AND street1.A != street2.B '
          'UNION '
          'SELECT street3.B AS company, (street1.d+street2.d+street3.d) AS distance '
          'FROM streets street1 '
          'JOIN streets street2 '
          'ON street1.B=street2.A '
          'JOIN streets street3 '
          'ON street2.B=street3.A '
          'WHERE street1.A=\'AUEB\' '
          'AND street1.d+street2.d+street3.d <= 10 '
          'AND street1.A != street2.B '
          'AND street2.A != street3.B '
          'AND street1.A != street3.B'
         )

for i, row in enumerate(cursor.execute(part3a)):
    print(row)

('DooHickey Collective', 7)
('DooHickey Corp', 9)
('Gadget Collective', 9)
('Gadget Corp', 6)
('Gizmo Corp', 9)
('Widget Industries', 10)


In [12]:
part3b = ('SELECT l.p1 AS p1, l.p2 AS p2, l.p3 AS p3, l.p4 AS p4, r.p5 AS p5, r.p6 AS p6, r.p7 AS p7, (l.distance + r.distance) AS distance '
          'FROM ('
              'SELECT NULL AS p1, NULL AS p2, A AS p3, B AS p4, d AS distance '
              'FROM streets '
              'WHERE B=\'AUEB\' '
              'UNION '
              'SELECT NULL AS p1, street1.A AS p2, street1.B as p3, street2.B AS p4, (street1.d+street2.d) AS distance '
              'FROM streets street1 '
              'JOIN streets street2 '
              'ON street1.B=street2.A '
              'WHERE street2.B=\'AUEB\' '
              'AND street1.A != street2.B '
              'UNION '
              'SELECT street1.A AS p1, street2.A AS p2, street3.A AS p3, street3.B AS p4, (street1.d+street2.d+street3.d) AS distance '
              'FROM streets street1 '
              'JOIN streets street2 '
              'ON street1.B=street2.A '
              'JOIN streets street3 '
              'ON street2.B=street3.A '
              'WHERE street3.B=\'AUEB\' '
              'AND street1.A != street2.B '
              'AND street2.A != street3.B '
              'AND street1.A != street3.B) l '
          'JOIN ('
              'SELECT A AS p4, B AS p5, NULL AS p6, NULL AS p7, d AS distance '
              'FROM streets '
              'WHERE A=\'AUEB\' '
              'UNION '
              'SELECT street1.A AS p4, street2.A AS p5, street2.B AS p6, NULL AS p7, (street1.d+street2.d) AS distance '
              'FROM streets street1 '
              'JOIN streets street2 '
              'ON street1.B=street2.A '
              'WHERE street1.A=\'AUEB\' '
              'AND street1.A != street2.B '
              'UNION '
              'SELECT street1.A AS p4, street2.A AS p5, street3.A AS p6, street3.B AS p7, (street1.d+street2.d+street3.d) AS distance '
              'FROM streets street1 '
              'JOIN streets street2 '
              'ON street1.B=street2.A '
              'JOIN streets street3 '
              'ON street2.B=street3.A '
              'WHERE street1.A=\'AUEB\' '
              'AND street1.A != street2.B '
              'AND street2.A != street3.B '
              'AND street1.A != street3.B) r '
          'ON l.p4=r.p4 '
          'WHERE l.distance+r.distance <= 15'
         )

#part3b = ('WITH path (A, B, d, hop) AS ('
#              'SELECT A, B, d, 1 '
#              'FROM streets '
#              'WHERE direction = \'F\' '
#              'UNION ALL '
#              'SELECT p2.A AS A, p2.B AS B, (p1.d+p2.d) AS d, p1.hop+1 '
#              'FROM path p1 '
#              'JOIN streets p2 '
#              'ON p1.B = p2.A '
#              'WHERE (p1.d+p2.d) <= 15 '
#              'AND hop <= 3'
#              'AND direction = \'F\' '
#          ') CYCLE A,B SET is_cycle TO 1 DEFAULT 0'
#          'SELECT * FROM path'
#         )

for i, row in enumerate(cursor.execute(part3b)):
    print(row)

(None, None, 'Gadget Corp', 'AUEB', 'DooHickey Collective', 'Gizmo Corp', None, 15)
(None, None, 'Gadget Corp', 'AUEB', 'DooHickey Collective', None, None, 13)
(None, None, 'DooHickey Collective', 'AUEB', 'DooHickey Collective', None, None, 14)
(None, None, 'Gadget Corp', 'AUEB', 'Gadget Collective', None, None, 15)
(None, None, 'Gadget Corp', 'AUEB', 'Gadget Corp', 'DooHickey Corp', None, 15)
(None, None, 'Gadget Corp', 'AUEB', 'Gadget Corp', None, None, 12)
(None, None, 'Gadget Collective', 'AUEB', 'Gadget Corp', None, None, 15)
(None, None, 'DooHickey Collective', 'AUEB', 'Gadget Corp', None, None, 13)
(None, 'Gizmo Corp', 'DooHickey Collective', 'AUEB', 'Gadget Corp', None, None, 15)
(None, 'DooHickey Corp', 'Gadget Corp', 'AUEB', 'Gadget Corp', None, None, 15)


We reenter the data in a second table, changing its name to streets2:

In [22]:
script_file = open('part3/Part 3.sql')
script = script_file.read()
commands = script.split(';')

for command in commands:
    if(command.isspace()==False and len(command) > 0):
        command = command.strip()
        command = command.replace('streets', 'streets2')
        cursor.execute(command)

In [14]:
part3c = ('WITH path (A, B, d, hop) AS ('
              'SELECT A, B, d, 1 '
              'FROM streets '
              'WHERE A = \'AUEB\' '
              'UNION ALL '
              'SELECT p1.A AS A, p1.B AS B, (p1.d+p2.d) AS d, p2.hop+1 '
              'FROM streets p1 '
              'JOIN path p2 '
              'ON p1.A = p2.B '
              'WHERE p1.B != p2.A '
          ') CYCLE A SET is_cycle TO 1 DEFAULT 0'
          'SELECT * FROM path '
         )
for i, row in enumerate(cursor.execute(part3c)):
    print(row)

('AUEB', 'DooHickey Collective', 7, 1, '0')
('AUEB', 'Gadget Corp', 6, 1, '0')
('AUEB', 'Gadget Collective', 9, 1, '0')
('DooHickey Collective', 'Gizmo Corp', 9, 2, '0')
('Gadget Corp', 'DooHickey Corp', 9, 2, '0')
('Gadget Corp', 'Gadget Industries', 15, 2, '0')
('Gadget Collective', 'GizmoWorks', 12, 2, '0')
('Gizmo Corp', 'GadgetWorks', 17, 3, '0')
('Gizmo Corp', 'Widget Industries', 10, 3, '0')
('DooHickey Corp', 'DooHickeyWorks', 12, 3, '0')
('Gadget Industries', 'Widget Corp', 20, 3, '0')
('DooHickey Corp', 'ThingCo', 15, 3, '0')
('Widget Industries', 'Gizmo Industries', 11, 4, '0')
('DooHickeyWorks', 'Thing Industries', 13, 4, '0')
('Thing Industries', 'Widget Collective', 15, 5, '0')
('Thing Industries', 'Gizmo Collective', 16, 5, '0')
('Thing Industries', 'GadgetCo', 22, 5, '0')
('Widget Collective', 'DooHickeyCo', 17, 6, '0')
('Widget Collective', 'WidgetWorks', 16, 6, '0')
('Widget Collective', 'GadgetCo', 22, 6, '0')
('GadgetCo', 'Widget Collective', 29, 6, '0')
('Widget Co

In [8]:
drop_streets = ('DROP TABLE streets')
cursor.execute(drop_streets)
drop_streets2 = ('DROP TABLE streets2')
cursor.execute(drop_streets2)